# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import numpy as np
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy import inspect

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
nltk.download(["punkt", "wordnet", "stopwords"])

import re
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dominik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dominik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dominik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# load data from database
engine = create_engine("sqlite:///DisasterResponse.db")
insp = inspect(engine)
print(insp.get_table_names())

['messages']


In [5]:
df = pd.read_sql_table("messages", engine)
X = df["message"]
y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    stop_words = stopwords.words("english")
    
    #tokenize
    words = word_tokenize (text)
    
    #stemming
    stemmed = [PorterStemmer().stem(w) for w in words]
    
    #lemmatizing
    words_lemmed = [WordNetLemmatizer().lemmatize(w) for w in stemmed if w not in stop_words]
    return words_lemmed

In [7]:
def ttokenize(text):
    """
    Tokenization function to process my text data
    
    Arguments:
        text - Messages that should be tokenized
    Output:
        clean_tokens - List of tokens extracted from the messages
    """
    
    url_regex = "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    urls = re.findall(url_regex, text)
    for url in urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_token)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline(steps = [
    ("count_vectorizer", CountVectorizer(tokenizer=tokenize)),
    ("transformer", TfidfTransformer()),
    ("classifier", MultiOutputClassifier(RandomForestClassifier(), n_jobs = 25))
])

In [9]:
pipeline

Pipeline(steps=[('count_vectorizer',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000218CA10E0D0>)),
                ('transformer', TfidfTransformer()),
                ('classifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier(),
                                       n_jobs=25))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [11]:
pipeline_fitted = pipeline.fit(X_train, y_train)
pipeline_fitted

Pipeline(steps=[('count_vectorizer',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000218CA10E0D0>)),
                ('transformer', TfidfTransformer()),
                ('classifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier(),
                                       n_jobs=25))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [36]:
y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

ValueError: WRITEBACKIFCOPY base is read-only

In [ ]:
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

In [ ]:
#y_pred = pipeline.predict(X_test)
#
#for i in range(36):
#    print(y_test.columns[i], ':')
#    print(classification_report(y_test.iloc[:,i], y_pred[:,i], target_names=category_names), '...................................................')

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.